## Statistical Modelling (Task-4)

In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
#load the data by pandas as dataframe
df = pd.read_csv(r'E:\KAIM 2\KAIM 2 Week 3\Week3\data\MachineLearningRating_v3.txt', sep='|')
df.head()

C:\Users\you\AppData\Local\Temp\ipykernel_252\304787155.py:2: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'E:\KAIM 2\KAIM 2 Week 3\Week3\data\MachineLearningRating_v3.txt', sep='|')


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [3]:
#check if there are missed values
df.isna().sum()

UnderwrittenCoverID               0
PolicyID                          0
TransactionMonth                  0
IsVATRegistered                   0
Citizenship                       0
LegalType                         0
Title                             0
Language                          0
Bank                         145961
AccountType                   40232
MaritalStatus                  8259
Gender                         9536
Country                           0
Province                          0
PostalCode                        0
MainCrestaZone                    0
SubCrestaZone                     0
ItemType                          0
mmcode                          552
VehicleType                     552
RegistrationYear                  0
make                            552
Model                           552
Cylinders                       552
cubiccapacity                   552
kilowatts                       552
bodytype                        552
NumberOfDoors               

##### interpretation of the above code

This code snippet checks for missing values in the DataFrame df using the isna().sum() method, which returns the count of NaN (not a number) values for each column. The output shows that several columns have missing values, with notable counts for Bank (145,961), AccountType (40,232), and Gender (9,536), among others. The column CustomValueEstimate has a significant number of missing entries (779,642), indicating potential issues with data completeness that may affect analyses or model training. 

Additionally, columns like WrittenOff, Rebuilt, and CrossBorder have extremely high counts of missing values, suggesting that these features may be unreliable or require careful handling before any further data processing or analysis. Understanding the extent of missing data is critical for determining how to address these gaps, whether through imputation, removal of affected records, or other strategies, to ensure the integrity and accuracy of any subsequent analyses or machine learning models.

In [6]:
# droping not specified Gender

df = df[df['Gender'] != 'Not specified']
df.dropna(subset=['NewVehicle'], inplace=True)
df_droped = df.drop(columns=['WrittenOff', 'NumberOfVehiclesInFleet', 'CrossBorder', 'Converted'])

the above code snippet is performing data cleaning on the DataFrame df to enhance the quality of the dataset for analysis. First, it filters out any rows where the Gender column is marked as 'Not specified', ensuring that the analysis focuses only on records with valid gender information. Next, it drops rows that have missing values in the NewVehicle column, which is critical if this variable is essential for subsequent analyses. Finally, the code creates a new DataFrame, df_dropped, by removing several columns (WrittenOff, NumberOfVehiclesInFleet, CrossBorder, and Converted) that may not be necessary for the analysis or could contain too many missing values. This step helps streamline the dataset by retaining only relevant features, facilitating more effective data analysis and modeling while reducing complexity and potential noise in the data.

In [10]:
df_droped.dropna(subset=['MaritalStatus', 'Bank'], inplace=True)

In [11]:
#new features catagory
features = ['LegalType', 'bodytype', 'Product', 'CoverGroup', 'Section', 'MaritalStatus']

In [8]:
##### one hot encoding
legaltype = pd.get_dummies(df.LegalType, dtype='int')
legaltype

,Close Corporation,Individual,Private company
28622,0,1,0
28623,0,1,0
28624,0,1,0
28625,0,1,0
28626,0,1,0
...,...,...,...
1000093,0,1,0
1000094,0,1,0
1000095,0,1,0
1000096,0,1,0


This code snippet uses the pd.get_dummies() function from the Pandas library to perform one-hot encoding on the LegalType column of the DataFrame df. One-hot encoding is a technique that converts categorical variables into a format that can be provided to machine learning algorithms to improve predictions. In this case, the values in LegalType are transformed into three binary columns: "Close Corporation," "Individual," and "Private company." Each row in the resulting DataFrame (legaltype) indicates the presence (1) or absence (0) of each legal type for the corresponding entry in the original DataFrame. The output shows that the majority of the entries are categorized as "Individual," with the other two categories showing no representation in the displayed rows. This transformation is essential for facilitating analyses and modeling, as many algorithms require numerical input, and one-hot encoding provides a way to represent categorical data while preserving the information contained in those categories.

In [12]:
#legal type vs total premium and total claim
df_legal_type = df[['TotalPremium', 'TotalClaims']]
merged_legal_type = pd.concat([df_legal_type, legaltype], axis='columns')
merged_legal_type.head()

,TotalPremium,TotalClaims,Close Corporation,Individual,Private company
28622,15.789474,0.0,0,1,0
28623,15.789474,0.0,0,1,0
28624,15.789474,0.0,0,1,0
28625,15.789474,0.0,0,1,0
28626,2.038421,0.0,0,1,0


the above code snippet creates a new DataFrame, merged_legal_type, by combining the TotalPremium and TotalClaims columns from the original DataFrame df with the one-hot encoded legaltype DataFrame created earlier. The pd.concat() function is used to concatenate these DataFrames along the columns, resulting in a dataset that directly associates the legal type of each entry with its corresponding total premium and total claims. The output displays the first few rows of this merged DataFrame, showing that the TotalPremium values vary, while TotalClaims is consistently zero for the displayed entries, indicating no claims have been made for those records. The presence of the binary indicators for each legal type (Close Corporation, Individual, and Private Company) allows for further analysis of how legal type relates to financial metrics, enabling insights into the risk profiles associated with different legal structures in relation to premiums paid and claims made. This merged dataset is critical for conducting deeper analyses, such as evaluating the impact of legal type on financial outcomes in the insurance context.

In [13]:
# spliting the data set
train, validate, test = np.split(merged_legal_type.sample(frac=1), [int(0.6*len(merged_legal_type)), int(0.8*len(merged_legal_type))])

c:\Users\you\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [14]:
from sklearn.linear_model import LinearRegression

In [15]:
model = LinearRegression()

In [16]:
# total premium
X = train.drop('TotalPremium', axis='columns')
y = train.TotalPremium

In [17]:
model.fit(X, y)

LinearRegression()

In [18]:
testx = test.drop('TotalPremium', axis='columns')
total_premium_pred = model.predict(testx)
total_premium_pred[:5]

array([53.53690212, 53.53690212, 53.53690212, 53.53690212, 94.15705834])

The above code snippet demonstrates the process of making predictions using a trained model in a machine learning context. It begins by creating a new DataFrame, testx, that excludes the TotalPremium column from the test DataFrame, which is presumably used as input features for the prediction. The model.predict(testx) function is then called to generate predictions for the total premium based on the remaining features in testx. The resulting array, total_premium_pred, contains the predicted total premiums for the first five entries, with values around 53.54 for several entries and one significantly higher prediction of approximately 94.16. The repeated value of 53.54 suggests that the model may be identifying underlying patterns or trends in the data that lead to similar predictions for multiple cases, while the variation in the last value indicates that some entries may have unique characteristics influencing the prediction. Overall, this snippet illustrates the application of a predictive model to estimate financial outcomes, a common task in data-driven decision-making in fields such as finance and insurance.

In [19]:
test.head()

,TotalPremium,TotalClaims,Close Corporation,Individual,Private company
51137,0.000000,0.0,0,1,0
977968,1.024561,0.0,0,1,0
951751,0.000000,0.0,0,1,0
863112,21.929825,0.0,0,1,0
986722,117.942982,0.0,0,0,1


The above code snippet displays the first few rows of the test DataFrame, which contains relevant features used for analysis or prediction. The DataFrame includes columns for TotalPremium, TotalClaims, and one-hot encoded categorical variables representing different legal types: "Close Corporation," "Individual," and "Private Company." 

The output shows that several entries have a TotalPremium of 0.00, indicating that no premium was paid for those cases, while others show varying premium amounts, such as approximately 21.93 and 117.94. Notably, the TotalClaims column consistently shows 0.0, suggesting that there have been no claims made for any of these entries, which might imply a lower risk profile for the insurance products associated with these records. 

The presence of the one-hot encoding indicates that the legal type is a factor in the dataset, allowing for analyses that explore how different legal structures correlate with premium amounts. Overall, this preview of the test DataFrame provides valuable insights into the structure and characteristics of the data being used for further modeling or evaluation.